<a href="https://colab.research.google.com/github/PaulToronto/DataCamp-Track---Machine-Learning-Scientist-in-Python/blob/main/1_4_Supervised_Learning_with_scikit_learn_Preprocessing_and_Pipelines.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1-4 Supervised Learning with scikit-learn - Preprocessing and Pipelines.

## Imports

In [2]:
import matplotlib.pyplot as plt

In [3]:
plt.style.use('ggplot')

## Data